<h1 align='center'> Data Cleaning </h1>

# Import Libraires and Jobs Data

In [1]:
import pandas as pd
import re

In [2]:
data_analyst_jobs = pd.read_csv('data/data_analyst_jobs_no_repeat.csv')
data_scientist_jobs = pd.read_csv('data/data_scientist_jobs_no_repeat.csv')
df = pd.concat([data_analyst_jobs, data_scientist_jobs], ignore_index=True)
df

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Investment Data Analyst,$74K - $118K (Glassdoor est.),Business Description\nDWS Group (DWS) is one o...,3.9,Deutsche Bank\n3.9,"Chicago, IL",10000+ Employees,1870,Company - Public,Banking & Lending,Financial Services,$10+ billion (USD)
1,CAT Risk Analyst,Employer Provided Salary:$67K - $110K,"Who Are We?\nTaking care of our customers, our...",3.9,"The Travelers Companies, Inc.\n3.9","Hartford, CT",10000+ Employees,1853,Company - Public,Insurance Carriers,Insurance,$10+ billion (USD)
2,Online Data Analyst,Employer Provided Salary:$12.10 Per Hour,About TELUS International AI Inc.:\nTELUS Inte...,3.8,TELUS International AI Inc.\n3.8,"Washington, DC",10000+ Employees,-1,Company - Public,Enterprise Software & Network Solutions,Information Technology,Unknown / Non-Applicable
3,Data Analyst,$63K - $94K (Glassdoor est.),Introduction to the job\nWe have an exciting o...,4.1,ASML\n4.1,"San Diego, CA",10000+ Employees,1984,Company - Public,Electronics Manufacturing,Manufacturing,$10+ billion (USD)
4,Data Specialist,-1,"StormTrap is seeking a passionate, organized a...",-1.0,StormTrap,Remote,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...
843,"Senior Data Scientist, Core Experience",$109K - $169K (Glassdoor est.),"Data, Research & Insights\nData Science\nDeliv...",4.3,Spotify\n4.3,"New York, NY",5001 to 10000 Employees,2006,Company - Public,Internet & Web Services,Information Technology,Unknown / Non-Applicable
844,Data Scientist (REMOTE),$48K - $79K (Glassdoor est.),The core data science team at Dick’s Sporting ...,3.8,DICK'S Sporting Goods\n3.8,"Coraopolis, PA",10000+ Employees,1948,Company - Public,Sporting Goods Stores,Retail & Wholesale,$5 to $10 billion (USD)
845,"Data Scientist, Engineering Analytics",$92K - $122K (Glassdoor est.),"United States, Georgia, Atlanta\nTechOps\n09-A...",4.3,Delta\n4.3,"Atlanta, GA",10000+ Employees,1928,Company - Public,"Airlines, Airports & Air Transportation",Transportation & Logistics,$10+ billion (USD)
846,Data Science Co-op (January - July 2023),Employer Provided Salary:$26.00 - $32.00 Per Hour,Help shape the future of Data Science across L...,3.9,Liberty Mutual Insurance\n3.9,Remote,10000+ Employees,1912,Company - Private,Insurance Carriers,Insurance,$10+ billion (USD)


# Drop Duplicate Rows

In [3]:
df.duplicated().value_counts()

False    845
True       3
dtype: int64

In [4]:
df.drop_duplicates(inplace=True, ignore_index=True)
df.duplicated().value_counts()

False    845
dtype: int64

# Rename Columns

In [5]:
# lowercase and replace space with _
col_name = list(df.columns)
df.columns = [name.lower().replace(' ', '_') for name in col_name]
list(df.columns)

['job_title',
 'salary_estimate',
 'job_description',
 'rating',
 'company_name',
 'location',
 'size',
 'founded',
 'type_of_ownership',
 'industry',
 'sector',
 'revenue']

# Data Cleaning

## ***`job_title`***
- Change the value of *`location`* to *Remote* if it appears in *`job_title`*.
- Create a new column *`seniority`* with 3 values : *NA*, *Low*, and *High*.
- Simplified *`job_title`* to 6 values : *NA*, *Data Analyst*, *Data Scientist*, *Data Engineer*, *ML Engineer*, and *Manager*.

In [6]:
df['job_title'][60:120]

60     Data Transformation Analyst II/Customer Suppor...
61                                      ESG Data Analyst
62                     Business Analyst - Internal Tools
63     Data Analyst/Medical Records Virtual Hiring Event
64                             Data Analyst - Contractor
65                                       Data Analyst II
66                         Data Visualization Specialist
67                                Marketing Data Analyst
68                                Data Reporting Analyst
69                   Data Analyst, Washington, DC/Remote
70                          Data Reporting Analyst (SQL)
71                         Business Analyst, Collections
72                                      Business Analyst
73                                    Business Analyst I
74                                  Data Quality Analyst
75                                Reporting Data Analyst
76                     Data & Telecommunications Analyst
77                            L

In [7]:
def remote(row):
    title = row['job_title'].lower()
    if 'remote' in title and 'not remote' not in title:
        row['location'] = 'Remote'
    return row

def seniority(title):
    title = title.lower()
    if 'sr' in title or 'senior' in title or 'mid' in title or 'experienced' in title or\
    'lead' in title or 'manager' in title or 'principal' in title or 'director' in title: 
        return 'High'
    elif 'jr' in title or 'junior' in title or 'entry' in title or 'associate' in title:
        return 'Low'
    else:
        return 'NA'
    
def title_simplify(title):
    title = title.lower()
    if 'analyst' in title:
        return 'Data Analyst'
    elif 'data scientist' in title or 'data science' in title:
        return 'Data Scientist'
    elif 'Data Engineer' in title:
        return 'data engineer'
    elif 'machine learning' in title or 'deep learning' in title or 'ai' in title or 'ml' in title:
        return 'ML Engineer'
    elif 'lead' in title or 'manager' in title or 'principal' in title or 'director' in title:
        return 'Manager'
    else:
        return 'NA'    

In [8]:
# change the value of location to Remote if it appears in job_title
df = df.apply(remote, axis=1)

# create a new column seniority with 3 values : NA, Low, and High   
df['seniority'] = df['job_title'].apply(seniority)

# simplified job_title to 6 values : NA, Data Analyst, Data Scientist, Data Engineer, ML Engineer, and Manager
df['job_title'] = df['job_title'].apply(title_simplify)

# check results
print(df['job_title'].value_counts())
print('\n')
print(df['seniority'].value_counts())

Data Scientist    365
Data Analyst      310
ML Engineer        99
NA                 57
Manager            14
Name: job_title, dtype: int64


NA      584
High    224
Low      37
Name: seniority, dtype: int64


## ***`salary_estimate`***
- *`salary_estimate`* is the target value, so remove the record when it is *-1*.
- Create a new column *`employer_provided_salary`*: *1* if the string contains *Employer Provided Salary*, otherwise *0*.
- Create a new column *`hourly_wage`*: *1* if the string contains *Per Hour*, otherwise *0*.
- Clean the string, keeping the format *XXX-XXX* only.
- Create new columns *`salary_min`* and *`salary_max`* from *(XXX)-(XXX)*. Turn into float type.
- Transfer the salary from hourly to yearly by multiplying 2 (unit in K) if *`hourly_wage`* is *1*.
- Create a new column *`salary_avg`* by averaging *`salary_min`* and *`salary_max`*.

In [9]:
df['salary_estimate'][:60]

0                         $74K - $118K (Glassdoor est.)
1                 Employer Provided Salary:$67K - $110K
2              Employer Provided Salary:$12.10 Per Hour
3                          $63K - $94K (Glassdoor est.)
4                                                    -1
5                                                    -1
6                          $49K - $76K (Glassdoor est.)
7                  Employer Provided Salary:$60K - $75K
8                                                    -1
9                                                    -1
10    Employer Provided Salary:$20.00 - $30.00 Per Hour
11                                                   -1
12                         $65K - $91K (Glassdoor est.)
13                                                   -1
14                 Employer Provided Salary:$70K - $80K
15                         $44K - $74K (Glassdoor est.)
16               Employer Provided Salary:$101K - $119K
17                         $56K - $83K (Glassdoo

In [10]:
# remove the record when salary is -1
df = df[df['salary_estimate'] != '-1']
# create employer_provided_salary: 1 if salary contains 'Employer', otherwise 0.
df['employer_provided_salary'] = df['salary_estimate'].apply(lambda x: 1 if 'Employer' in x else 0)
# create hourly_wage: 1 if salary contains 'Per Hour', otherwise 0.
df['hourly_wage'] = df['salary_estimate'].apply(lambda x: 1 if 'Per Hour' in x else 0)


    # extract formats $___K,  $__.__,  $___K - $___K,  $__.__ - $__.__
pattern = r'(\$[-$\d.K ]*[K\d])'
result = (df['salary_estimate'].str.extract(pattern).squeeze()
    # remove $, K, and space
    .str.replace('[$K ]', '', regex=True)
    # transform XXX -> XXX-XXX
    .apply(lambda x: x if '-' in x else x + '-' + x)
    # create salary_min and salary_max     
    .str.extract(r'(?P<salary_min>.*)-(?P<salary_max>.*)')
    .astype(float))


salary = ['salary_min', 'salary_max']
# transfer the salary from hourly to yearly by multiplying 2 (unit in K) if hourly_wage is 1
result = (pd.concat([result, df[['hourly_wage']]], axis=1)    
    .apply(lambda x: x[salary]*2 if x['hourly_wage'] == 1 else x[salary], axis=1))
df = pd.concat([df, result], axis=1)

# create salary_avg by averaging salary_min and salary_max
df['salary_avg'] = (df['salary_min'] + df['salary_max']) / 2

df.drop(['salary_estimate'], axis=1, inplace=True)

# check results
df.loc[:, 'employer_provided_salary':'salary_avg'][:60]

,employer_provided_salary,hourly_wage,salary_min,salary_max,salary_avg
0,0,0,74.00,118.00,96.00
1,1,0,67.00,110.00,88.50
2,1,1,24.20,24.20,24.20
3,0,0,63.00,94.00,78.50
6,0,0,49.00,76.00,62.50
7,1,0,60.00,75.00,67.50
10,1,1,40.00,60.00,50.00
12,0,0,65.00,91.00,78.00
14,1,0,70.00,80.00,75.00
15,0,0,44.00,74.00,59.00


## ***`job_description`***
- Create a new column *`description_len`* to count the length of the job description.
- Create *`skill_XX`* columns: *1* if *`job_description`* mentions the specific skill, otherwise *0*. Skills include:
    - Big Data (Spark or Hadoop)
    - Cloud Computing (AWS or Azure)    
    - Excel
    - Machine Learning or Deep Learning
    - R or Python
    - SQL
    - Visualization Tools (Tableau, Power BI)    

In [11]:
# create description_len to count the length of the job description
df['description_len'] = df['job_description'].apply(len) 

# create skill_XX: 1 if job_description mentions the specific skill, otherwise 0
# Spark or Hadoop
df['skill_bigdata'] = df['job_description'].apply(lambda x: 1 if re.search('Spark|Hadoop', x) else 0)
# AWS or Azure
df['skill_cloud'] = df['job_description'].apply(lambda x: 1 if re.search('AWS|Azure', x) else 0)
# Excel
df['skill_excel'] = df['job_description'].apply(lambda x: 1 if re.search('\WExcel\W', x) else 0)
# machine learning or deep learning
df['skill_ml'] = df['job_description'].apply(lambda x: 1 if re.search('machine learning|deep learning', x.lower()) else 0)
# R or Python
df['skill_rpython'] = df['job_description'].apply(lambda x: 1 if re.search('\W(R|Python)\W', x) else 0)
# SQL
df['skill_sql'] = df['job_description'].apply(lambda x: 1 if re.search('SQL', x) else 0)
# Tableau, Power BI
df['skill_viztool'] = df['job_description'].apply(lambda x: 1 if re.search('Tableau|PowerBI|Power BI', x) else 0)

df.drop(['job_description'], axis=1, inplace=True)

# check results
df.loc[:, 'description_len':'skill_viztool']

,description_len,skill_bigdata,skill_cloud,skill_excel,skill_ml,skill_rpython,skill_sql,skill_viztool
0,3932,0,0,1,0,1,1,0
1,5745,0,0,0,0,0,1,0
2,2380,0,0,0,1,0,0,0
3,5177,0,0,1,0,0,1,1
6,1614,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
840,4602,0,0,0,0,1,1,0
841,1961,0,1,0,1,1,1,0
842,2333,0,1,0,1,1,1,0
843,3802,0,0,0,1,1,0,0


## ***`rating`***

In [12]:
df['rating'].value_counts()

 3.9    82
 3.8    69
 4.0    68
 4.2    56
 4.1    54
-1.0    47
 4.4    44
 3.7    39
 3.6    37
 3.5    29
 4.3    26
 3.3    25
 3.4    21
 5.0    18
 4.5    15
 3.2     9
 4.6     9
 3.1     8
 4.8     7
 4.7     6
 2.8     3
 3.0     3
 1.0     2
 4.9     2
 2.7     1
 2.6     1
 2.9     1
 2.2     1
 2.0     1
Name: rating, dtype: int64

## ***`company_name`***
- Remove the new line and rating

In [13]:
print(df['company_name'][:60])

# remove new line and rating if rating exists
df['company_name'] = df.apply(lambda x: x['company_name'] if x['rating'] < 0 else x['company_name'][:-4], axis=1)

# check results
print('\n')
df['company_name'][:60]

0                                    Deutsche Bank\n3.9
1                    The Travelers Companies, Inc.\n3.9
2                      TELUS International AI Inc.\n3.8
3                                             ASML\n4.1
6                             Humboldt Park Health\n2.2
7                            Prosperitus Solutions\n1.0
10                                         Confidential
12                                         Emerson\n3.8
14                              World Services LLC\n4.7
15                                            ASML\n4.1
16                                       U.S. Bank\n3.8
17                                  AccruePartners\n4.2
18                                            IKEA\n3.7
19                             Qi Venture Partners\n5.0
20                            Norwood Development Group
21                   Shellpoint Mortgage Servicing\n3.2
22                                        SoCalGas\n4.4
23                 Benefit Allocation Systems, L

0                                     Deutsche Bank
1                     The Travelers Companies, Inc.
2                       TELUS International AI Inc.
3                                              ASML
6                              Humboldt Park Health
7                             Prosperitus Solutions
10                                     Confidential
12                                          Emerson
14                               World Services LLC
15                                             ASML
16                                        U.S. Bank
17                                   AccruePartners
18                                             IKEA
19                              Qi Venture Partners
20                        Norwood Development Group
21                    Shellpoint Mortgage Servicing
22                                         SoCalGas
23                  Benefit Allocation Systems, LLC
24             Moody Neuro Rehabilitation Institute
25          

## ***`location`***
- Keeps the *Remote* and the 2-digit state abbreviations
- Fix values which are not mentioned above

In [14]:
df['location'].value_counts()

Remote               152
New York, NY          56
San Francisco, CA     28
Seattle, WA           19
Austin, TX            18
                    ... 
Columbia, MD           1
Boise, ID              1
Duluth, MN             1
Dayton, OH             1
Chantilly, VA          1
Name: location, Length: 235, dtype: int64

In [15]:
# get the string after ',', remain the same if ',' doesn't exist
df['location'] = df['location'].apply(lambda x: x.split(', ')[1] if x.find(',') != -1 else x)
df['location'].value_counts()

Remote              152
CA                  115
NY                   62
TX                   42
VA                   32
WA                   30
MA                   28
IL                   20
NJ                   18
GA                   14
FL                   13
PA                   13
MD                   12
CT                   11
NC                   11
TN                   10
MI                    9
CO                    8
OH                    6
MN                    6
DE                    6
Manhattan             6
MO                    5
DC                    5
UT                    4
SC                    4
IN                    4
WI                    4
United States         3
AZ                    2
Florida               2
IA                    2
RI                    2
Texas                 1
OK                    1
Colorado              1
Alabama               1
Pennsylvania          1
Alaska                1
LA                    1
AR                    1
NE              

In [16]:
# fix errors
fix = {'Manhattan': 'NY',
       'United States': 'Remote',
       'Florida': 'FL',
       'Texas': 'TX',
       'Colorado': 'CO',
       'Alabama': 'AL',
       'Pennsylvania' : 'PA',
       'Alaska': 'AK',
       'Washington State': 'WA',
       'Ohio': 'OH',
       'South Carolina': 'SC',
       'Utah': 'UT'
       }
df['location'] = df['location'].replace(fix)

# check results
df['location'].value_counts()

Remote    155
CA        115
NY         68
TX         43
VA         32
WA         31
MA         28
IL         20
NJ         18
FL         15
GA         14
PA         14
MD         12
NC         11
CT         11
TN         10
CO          9
MI          9
OH          7
MN          6
DE          6
MO          5
DC          5
UT          5
SC          5
WI          4
IN          4
AL          2
AZ          2
IA          2
RI          2
NE          1
AR          1
LA          1
WV          1
AK          1
OK          1
KS          1
NM          1
WY          1
ID          1
KY          1
OR          1
ME          1
MS          1
Name: location, dtype: int64

## ***`size`***
- *-1* &rarr; *Unknown*

In [17]:
print(df['size'].value_counts())

df['size'] = df['size'].str.replace('-1', 'Unknown')

# check results
print('\n')
df['size'].value_counts()

10000+ Employees           267
1001 to 5000 Employees     106
51 to 200 Employees         61
201 to 500 Employees        58
1 to 50 Employees           57
5001 to 10000 Employees     41
501 to 1000 Employees       37
-1                          31
Unknown                     26
Name: size, dtype: int64




10000+ Employees           267
1001 to 5000 Employees     106
51 to 200 Employees         61
201 to 500 Employees        58
Unknown                     57
1 to 50 Employees           57
5001 to 10000 Employees     41
501 to 1000 Employees       37
Name: size, dtype: int64

## ***`founded`***
- 2022 - founded year

In [18]:
print(df['founded'])

df['age'] = df['founded'].apply(lambda x: 2022-x if x != -1 else x)
df.drop(['founded'], axis=1, inplace=True)

# show results
print('\n')
df['age']

0      1870
1      1853
2        -1
3      1984
6      1894
       ... 
840    2006
841    1948
842    1928
843    1912
844    2016
Name: founded, Length: 684, dtype: int64




0      152
1      169
2       -1
3       38
6      128
      ... 
840     16
841     74
842     94
843    110
844      6
Name: age, Length: 684, dtype: int64

## ***`type_of_ownership`***
- *-1* &rarr; *Unknown*

In [19]:
print(df['type_of_ownership'].value_counts())

df['type_of_ownership'] = df['type_of_ownership'].str.replace('-1', 'Unknown')

# check results
print('\n')
df['type_of_ownership'].value_counts()

Company - Private                 279
Company - Public                  263
Subsidiary or Business Segment     33
-1                                 31
Nonprofit Organization             26
Government                         17
College / University               13
Unknown                             6
Hospital                            5
Contract                            4
Self-employed                       4
Private Practice / Firm             3
Name: type_of_ownership, dtype: int64




Company - Private                 279
Company - Public                  263
Unknown                            37
Subsidiary or Business Segment     33
Nonprofit Organization             26
Government                         17
College / University               13
Hospital                            5
Contract                            4
Self-employed                       4
Private Practice / Firm             3
Name: type_of_ownership, dtype: int64

## ***`industry`***

In [20]:
df['industry'].value_counts()[:60]

HR Consulting                              71
-1                                         70
Internet & Web Services                    54
Information Technology Support Services    50
Health Care Services & Hospitals           49
Banking & Lending                          34
Computer Hardware Development              28
Staffing & Subcontracting                  25
Insurance Carriers                         24
Business Consulting                        23
Biotech & Pharmaceuticals                  22
Enterprise Software & Network Solutions    20
National Agencies                          14
Colleges & Universities                    13
Energy & Utilities                         13
General Merchandise & Superstores          12
Home Furniture & Housewares Stores         10
Investment & Asset Management              10
Telecommunications Services                 9
Insurance Agencies & Brokerages             9
Software Development                        8
Aerospace & Defense               

## ***`sector`***

In [21]:
df['sector'].value_counts()

Information Technology                         160
Human Resources & Staffing                      96
-1                                              70
Financial Services                              55
Healthcare                                      49
Retail & Wholesale                              34
Insurance                                       33
Management & Consulting                         33
Manufacturing                                   30
Pharmaceutical & Biotechnology                  23
Media & Communication                           19
Government & Public Administration              16
Education                                       14
Energy, Mining & Utilities                      13
Telecommunications                              10
Aerospace & Defense                              8
Transportation & Logistics                       8
Construction, Repair & Maintenance Services      3
Arts, Entertainment & Recreation                 3
Restaurants & Food Service     

## ***`revenue`***
- *-1* &rarr; *Unknown / Non-Applicable*

In [22]:
print(df['revenue'].value_counts())

df['revenue'] = df['revenue'].str.replace('-1', 'Unknown / Non-Applicable')

# show results
print('\n')
df['revenue'].value_counts()

Unknown / Non-Applicable            191
$10+ billion (USD)                  177
$1 to $5 billion (USD)               62
$100 to $500 million (USD)           59
$25 to $100 million (USD)            51
$5 to $10 billion (USD)              39
-1                                   31
$5 to $25 million (USD)              29
$500 million to $1 billion (USD)     27
$1 to $5 million (USD)               13
Less than $1 million (USD)            5
Name: revenue, dtype: int64




Unknown / Non-Applicable            222
$10+ billion (USD)                  177
$1 to $5 billion (USD)               62
$100 to $500 million (USD)           59
$25 to $100 million (USD)            51
$5 to $10 billion (USD)              39
$5 to $25 million (USD)              29
$500 million to $1 billion (USD)     27
$1 to $5 million (USD)               13
Less than $1 million (USD)            5
Name: revenue, dtype: int64

# Output Cleaned Data

In [23]:
df

,job_title,rating,company_name,location,size,type_of_ownership,industry,sector,revenue,seniority,...,salary_avg,description_len,skill_bigdata,skill_cloud,skill_excel,skill_ml,skill_rpython,skill_sql,skill_viztool,age
0,Data Analyst,3.9,Deutsche Bank,IL,10000+ Employees,Company - Public,Banking & Lending,Financial Services,$10+ billion (USD),NA,...,96.0,3932,0,0,1,0,1,1,0,152
1,Data Analyst,3.9,"The Travelers Companies, Inc.",CT,10000+ Employees,Company - Public,Insurance Carriers,Insurance,$10+ billion (USD),NA,...,88.5,5745,0,0,0,0,0,1,0,169
2,Data Analyst,3.8,TELUS International AI Inc.,DC,10000+ Employees,Company - Public,Enterprise Software & Network Solutions,Information Technology,Unknown / Non-Applicable,NA,...,24.2,2380,0,0,0,1,0,0,0,-1
3,Data Analyst,4.1,ASML,CA,10000+ Employees,Company - Public,Electronics Manufacturing,Manufacturing,$10+ billion (USD),NA,...,78.5,5177,0,0,1,0,0,1,1,38
6,Data Analyst,2.2,Humboldt Park Health,IL,501 to 1000 Employees,Hospital,Health Care Services & Hospitals,Healthcare,$100 to $500 million (USD),NA,...,62.5,1614,0,0,0,0,0,0,0,128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
840,Data Scientist,4.3,Spotify,NY,5001 to 10000 Employees,Company - Public,Internet & Web Services,Information Technology,Unknown / Non-Applicable,High,...,139.0,4602,0,0,0,0,1,1,0,16
841,Data Scientist,3.8,DICK'S Sporting Goods,Remote,10000+ Employees,Company - Public,Sporting Goods Stores,Retail & Wholesale,$5 to $10 billion (USD),NA,...,63.5,1961,0,1,0,1,1,1,0,74
842,Data Scientist,4.3,Delta,GA,10000+ Employees,Company - Public,"Airlines, Airports & Air Transportation",Transportation & Logistics,$10+ billion (USD),NA,...,107.0,2333,0,1,0,1,1,1,0,94
843,Data Scientist,3.9,Liberty Mutual Insurance,Remote,10000+ Employees,Company - Private,Insurance Carriers,Insurance,$10+ billion (USD),NA,...,58.0,3802,0,0,0,1,1,0,0,110


In [24]:
df.to_csv('data/glassdoor_jobs_cleaned.csv', index=False)